In [3]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [4]:
usuarios = pd.read_csv('events_up_to_01062018.csv',low_memory=False)
training = pd.read_csv('labels_training_set.csv',low_memory=False)

In [5]:
usuarios['timestamp'] = pd.to_datetime(usuarios['timestamp'], errors = 'coerce', format= '%Y-%m-%d')
usuarios['event'] = pd.Categorical(usuarios['event'])
usuarios['url'] = pd.Categorical(usuarios['url'])
usuarios['model'] = pd.Categorical(usuarios['model'])
usuarios['condition'] = pd.Categorical(usuarios['condition'])
usuarios['storage'] = pd.Categorical(usuarios['storage'])
usuarios['color'] = pd.Categorical(usuarios['color'])
usuarios['staticpage'] = pd.Categorical(usuarios['staticpage'])
usuarios['campaign_source'] = pd.Categorical(usuarios['campaign_source'])
usuarios['search_engine'] = pd.Categorical(usuarios['search_engine'])
usuarios['channel'] = pd.Categorical(usuarios['channel'])
usuarios['new_vs_returning'] = pd.Categorical(usuarios['new_vs_returning'])
usuarios['city'] = pd.Categorical(usuarios['city'])
usuarios['region'] = pd.Categorical(usuarios['region'])
usuarios['country'] = pd.Categorical(usuarios['country'])
usuarios['device_type'] = pd.Categorical(usuarios['device_type'])
usuarios['screen_resolution'] = pd.Categorical(usuarios['screen_resolution'])
usuarios['operating_system_version'] = pd.Categorical(usuarios['operating_system_version'])
usuarios['browser_version'] = pd.Categorical(usuarios['browser_version'])
usuarios['month'] = usuarios['timestamp'].dt.month
usuarios['day'] = usuarios['timestamp'].dt.day
usuarios['hora'] = usuarios['timestamp'].dt.hour

In [9]:
ultima_conexion = usuarios.groupby('person').agg({'timestamp':'max'})
ultima_conexion.head()

,timestamp
person,
0008ed71,2018-05-17 16:28:37
00091926,2018-05-31 19:52:03
00091a7a,2018-03-26 14:56:58
000ba417,2018-05-26 13:09:22
000c79fe,2018-05-29 00:38:07


In [10]:
set_entrenamiento = pd.merge(training, ultima_conexion, on='person',how='left')
set_entrenamiento.head(3)

,person,label,timestamp
0,0566e9c1,0,2018-05-31 16:31:24
1,6ec7ee77,0,2018-05-28 02:25:24
2,abe7a2fb,0,2018-05-29 23:39:14


In [11]:
set_entrenamiento['label'].mean()

0.05047903574739879

In [12]:
set_entrenamiento['label'].value_counts()

0    18434
1      980
Name: label, dtype: int64

In [13]:
print(len(training[training['label'] == 0]))
print(len(training[training['label'] == 1]))

18434
980


In [46]:
#Vemos que la cantidad de labels en 0 es muy superior a los 1. 
#Para balancear el set elegiremos 980 usuarios de los que tienen label 0

In [14]:
set_reducido_0 = set_entrenamiento[set_entrenamiento['label'] == 0].sample(n=980)
set_entrenamiento = set_entrenamiento[set_entrenamiento['label'] == 1].append(set_reducido_0)
set_entrenamiento.shape

(1960, 3)

In [16]:
set_entrenamiento = set_entrenamiento[['person','label']]
set_entrenamiento.head()

,person,label
5,db2c4d27,1
25,8123457d,1
46,e4b02ea2,1
48,d8001b23,1
54,7a472832,1


In [19]:
set_entrenamiento.describe()

,label
count,1960.000000
mean,0.500000
std,0.500128
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [20]:
set_entrenamiento.to_csv('../modelos/set_entrenamiento.csv',index=False)